[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

Welcome to my Covid-19 Dashboard! This dashboard represents a few interesting features that you can interact with as per your interest :)

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={"caseadmissions.json"}

## the data frame

This data frame represents the data the data retrieved via an API from the .gov covid-19 database.

In [4]:
def wrangle_data(jsondata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    with open("caseadmissions.json", "rt") as INFILE:
        data=json.load(INFILE)
        datalist=data['data']
        dates=[dictionary['date'] for dictionary in datalist]
        dates.sort()
        
        startdate=parse_date(dates[0])
        enddate=parse_date(dates[-1])
        index=pd.date_range(startdate, enddate, freq='D')
        caseadmissionsdf=pd.DataFrame(index=index, columns=['cases', 'hospital'])
        
        for entry in datalist:
            date=parse_date(entry['date'])
            for column in ['cases', 'hospital']:
                if pd.isna(caseadmissionsdf.loc[date, column]):
                    value= float(entry[column]) if entry[column]!=None else 0.0
                    caseadmissionsdf.loc[date, column]=value
        caseadmissionsdf.fillna(0.0, inplace=True)
    return caseadmissionsdf

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
caseadmissionsdf = wrangle_data(jsondata)

In [5]:
caseadmissionsdf

,cases,hospital
2020-01-31,2.0,0.0
2020-02-01,0.0,0.0
2020-02-02,0.0,0.0
2020-02-03,0.0,0.0
2020-02-04,0.0,0.0
...,...,...
2022-09-07,0.0,608.0
2022-09-08,0.0,588.0
2022-09-09,0.0,490.0
2022-09-10,0.0,451.0


In [6]:
#caseadmissionsdf.plot()

In [7]:
#caseadmissionsdf.plot(logy=True)

## Download current data

In [8]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 

def access_api(api_button):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = ['areaType=overview']
    structure = {"date" : "date","cases": "newCasesByPublishDate","hospital": "newAdmissions"}
    api = Cov19API(filters=filters, structure=structure)
    caseadmissions=api.get_json()
    return caseadmissions # return data read from the API
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again.   


In [9]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    global caseadmissionsdf
    apidata=access_api(button)
    #caseadmissionsdf = wrangle_data(jsondata)
    # wrangle the data and overwrite the dataframe for plotting
    casadmissionsdf=wrangle_data(apidata)
    #caseadmissionsdf.plot()
    #caseadmissionsdf.plot(logy=True)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph() # forget about this in the beginning
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

apibutton=wdg.Button(description='Refresh data',disabled=False,button_style='', # 'success', 'info', 'warning', 'danger' or ''
                     tooltip='Click to download current Public Health England data', icon='download')# (FontAwesome names without the `fa-` prefix)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [10]:
#caseadmissions_df=pd.read_pickle("caseadmissionsdf.pkl")

In [14]:
series=wdg.SelectMultiple(
    options=['cases', 'hospital'],
    value=['cases', 'hospital'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

def refresh_graph():
    current=scale.value
    other=[x for x in scale.options if x!=current][0]
    scale.value=other
    scale.value=current

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale]) # can use vbox to get them one below another

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        caseadmissionsdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

In [15]:
refresh_graph()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*